<a href="https://colab.research.google.com/github/skyadav1989/python-collab-notebooks/blob/main/Advanced_Python_Concepts_All_Topics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🐍 Advanced Python Concepts
### A Complete Reference with Real-World Examples

> **21 topics** · Production-grade code · Google Colab ready

---

| # | Topic | # | Topic |
|---|-------|---|-------|
| 1 | Metaclasses | 12 | `__slots__` |
| 2 | Descriptors | 13 | Abstract Base Classes |
| 3 | Context Managers | 14 | Monkey Patching |
| 4 | Decorators | 15 | Function Closures |
| 5 | Generators & Coroutines | 16 | Introspection & Reflection |
| 6 | Asyncio (async/await) | 17 | Dynamic Attribute Access |
| 7 | Multithreading | 18 | Data Model & Dunder Methods |
| 8 | Multiprocessing | 19 | C Extensions (Cython/CFFI) |
| 9 | Global Interpreter Lock | 20 | Bytecode & Python VM |
| 10 | Memory Management & GC | 21 | Profiling & Performance |
| 11 | Weak References | | |

---


In [ ]:
# ✅ Run this first — standard library only, no installs needed
import sys
print(f"Python {sys.version}")
print("All cells use stdlib only — no pip installs required!")


---
## 🏗️ Topic 1: Metaclasses
`────────────────────────────────────────────────────────────`


### What is it?
A **metaclass** is the *class of a class*. Just as an instance is built by its class,
a class is built by its metaclass. By default every Python class uses `type` as its metaclass.

When Python executes a `class` statement it calls `type.__new__(mcs, name, bases, namespace)`.
Overriding this lets you inspect or modify the class **before it ever exists**.

### Real-World Use Case
> **Django ORM** — model fields are discovered automatically, subclasses are registered
> in an internal registry, and database table names are derived — all without the developer
> calling any registration function explicitly.

### How it works
```
User defines class  ──►  Python calls metaclass.__new__()  ──►  class object created
```


In [ ]:
# ── Example: Auto-registering API endpoint router ────────────────────────────

class EndpointMeta(type):
    """Metaclass that auto-populates a URL registry."""
    registry = {}

    def __new__(mcs, name, bases, namespace):
        cls = super().__new__(mcs, name, bases, namespace)
        route = namespace.get('route')
        if route:                          # only register concrete endpoints
            mcs.registry[route] = cls
        return cls


class BaseEndpoint(metaclass=EndpointMeta):
    """All API endpoints inherit from this."""


class UserEndpoint(BaseEndpoint):
    route = '/users'
    def handle(self, request): return {'users': ['Alice', 'Bob']}


class ProductEndpoint(BaseEndpoint):
    route = '/products'
    def handle(self, request): return {'products': ['Widget', 'Gadget']}


class OrderEndpoint(BaseEndpoint):
    route = '/orders'
    def handle(self, request): return {'orders': [101, 102]}


# ── Router is auto-populated — zero manual registration ──────────────────────
print("Registered routes:", list(EndpointMeta.registry.keys()))

def dispatch(route, request={}):
    handler_cls = EndpointMeta.registry.get(route)
    if handler_cls:
        return handler_cls().handle(request)
    return {'error': '404 Not Found'}

print(dispatch('/users'))
print(dispatch('/products'))
print(dispatch('/missing'))


### Key Insight
Every time a subclass of `BaseEndpoint` is **defined**, `EndpointMeta.__new__` fires
and populates the registry — no decorator, no manual call needed.

### When to use
- Framework-level concerns (ORM field discovery, plugin registries, interface enforcement)
- For simpler needs prefer `__init_subclass__` or class decorators — less cognitive load


---
## 🔌 Topic 2: Descriptors
`────────────────────────────────────────────────────────────`


### What is it?
A **descriptor** is any object that defines `__get__`, `__set__`, or `__delete__`.
Descriptors are the engine behind `@property`, `classmethod`, `staticmethod`, and ORM fields.

| Type | Methods | Priority |
|------|---------|----------|
| **Data descriptor** | `__get__` + `__set__` or `__delete__` | Beats instance `__dict__` |
| **Non-data descriptor** | `__get__` only | Overridden by instance attributes |

### Real-World Use Case
> **Django model fields** — `CharField`, `IntegerField` etc. are descriptors that
> intercept attribute access to handle validation, lazy loading, and serialisation.


In [ ]:
# ── Example: Validated field descriptor (mini-Django) ────────────────────────

class TypedField:
    """Descriptor that enforces a type and optional numeric range."""

    def __init__(self, field_type, min_val=None, max_val=None):
        self.field_type = field_type
        self.min_val    = min_val
        self.max_val    = max_val
        self.private_name = None   # set by __set_name__

    def __set_name__(self, owner, name):
        """Called automatically when the owning class is created."""
        self.private_name = f'_{name}'

    def __get__(self, obj, objtype=None):
        if obj is None:            # accessed on the class itself
            return self
        return getattr(obj, self.private_name, None)

    def __set__(self, obj, value):
        if not isinstance(value, self.field_type):
            raise TypeError(
                f'Expected {self.field_type.__name__}, got {type(value).__name__}'
            )
        if self.min_val is not None and value < self.min_val:
            raise ValueError(f'Value {value!r} is below minimum {self.min_val}')
        if self.max_val is not None and value > self.max_val:
            raise ValueError(f'Value {value!r} exceeds maximum {self.max_val}')
        setattr(obj, self.private_name, value)


class Product:
    name  = TypedField(str)
    price = TypedField(float, min_val=0.0)
    stock = TypedField(int,   min_val=0, max_val=10_000)

    def __init__(self, name, price, stock):
        self.name  = name
        self.price = price
        self.stock = stock

    def __repr__(self):
        return f'Product({self.name!r}, ${self.price}, qty={self.stock})'


# ── Happy path ────────────────────────────────────────────────────────────────
p = Product('Widget', 9.99, 100)
print(p)
print('Price:', p.price)

# ── Validation errors ────────────────────────────────────────────────────────
for attempt in [
    lambda: Product('Widget', -1.0,  100),   # negative price
    lambda: Product(42,        9.99, 100),   # wrong type for name
    lambda: Product('Widget',  9.99, 99999), # stock too high
]:
    try:
        attempt()
    except (TypeError, ValueError) as e:
        print(f'  ✗ {type(e).__name__}: {e}')


---
## 🔒 Topic 3: Context Managers
`────────────────────────────────────────────────────────────`


### What is it?
Context managers manage resources by **guaranteeing setup and teardown** via the `with`
statement, even if an exception occurs. They implement `__enter__` and `__exit__`
(class-based) or use `@contextlib.contextmanager` (generator-based).

### Real-World Use Cases
> Database transactions · File locking · Temporary directories · Mock patching in tests
> · Distributed transaction management · Timer context managers for benchmarking

### Protocol
```python
with SomeContextManager() as value:
    # __enter__ ran, value returned
    ...
# __exit__ always runs here — even after exceptions
```


In [ ]:
import sqlite3
from contextlib import contextmanager

# ── Class-based approach ──────────────────────────────────────────────────────

class DatabaseConnection:
    """Context manager that wraps a SQLite connection in a transaction."""

    def __init__(self, db_path: str):
        self.db_path = db_path
        self.conn    = None

    def __enter__(self):
        self.conn = sqlite3.connect(self.db_path)
        self.conn.execute('BEGIN')
        print('  [DB] Transaction started')
        return self.conn.cursor()

    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is None:
            self.conn.commit()
            print('  [DB] ✓ Committed')
        else:
            self.conn.rollback()
            print(f'  [DB] ✗ Rolled back — {exc_type.__name__}: {exc_val}')
        self.conn.close()
        return False   # don't suppress exceptions


# ── Generator-based approach with contextlib ──────────────────────────────────

@contextmanager
def managed_transaction(db_path: str):
    conn = sqlite3.connect(db_path)
    try:
        conn.execute('BEGIN')
        yield conn.cursor()
        conn.commit()
        print('  [CTX] ✓ Committed')
    except Exception:
        conn.rollback()
        print('  [CTX] ✗ Rolled back')
        raise
    finally:
        conn.close()


# ── Demo: successful transaction ─────────────────────────────────────────────
print("=== Successful transaction ===")
with DatabaseConnection(':memory:') as cur:
    cur.execute('CREATE TABLE items (id INTEGER, name TEXT)')
    cur.execute("INSERT INTO items VALUES (1, 'Alpha')")
    cur.execute("INSERT INTO items VALUES (2, 'Beta')")

# ── Demo: failed transaction (auto-rollback) ──────────────────────────────────
print("\n=== Failed transaction (auto-rollback) ===")
try:
    with DatabaseConnection(':memory:') as cur:
        cur.execute('CREATE TABLE x (id INTEGER)')
        cur.execute("INSERT INTO x VALUES (1)")
        raise RuntimeError("Simulated mid-transaction failure!")
except RuntimeError:
    pass

# ── Demo: contextmanager version ─────────────────────────────────────────────
print("\n=== contextmanager version ===")
with managed_transaction(':memory:') as cur:
    cur.execute('CREATE TABLE items (id INTEGER, name TEXT)')
    cur.execute("INSERT INTO items VALUES (1, 'Gamma')")


---
## 🎨 Topic 4: Decorators
`────────────────────────────────────────────────────────────`


### What is it?
A **decorator** is a callable that takes a function (or class) as input and returns a
replacement. They add cross-cutting concerns — caching, logging, rate-limiting, auth —
**without modifying** the original function.

### Real-World Use Cases
> `@app.route` in Flask · `@router.get` in FastAPI · `@login_required` in Django
> · `@retry` in microservices · `@cache` for memoisation · `@pytest.fixture`

### Anatomy
```
@decorator          ← sugar for:  func = decorator(func)
def func(): ...
```


In [ ]:
import time, functools, logging

logging.basicConfig(level=logging.WARNING, format='%(message)s')
logger = logging.getLogger(__name__)

# ── 1. Retry with exponential back-off ───────────────────────────────────────

def retry(max_attempts=3, delay=0.1, backoff=2.0, exceptions=(Exception,)):
    """Decorator factory: retries a function on transient failures."""
    def decorator(func):
        @functools.wraps(func)   # preserves __name__, __doc__, __annotations__
        def wrapper(*args, **kwargs):
            wait = delay
            for attempt in range(1, max_attempts + 1):
                try:
                    return func(*args, **kwargs)
                except exceptions as exc:
                    if attempt == max_attempts:
                        raise
                    print(f'  ⚠ Attempt {attempt}/{max_attempts} failed ({exc}). '
                          f'Retrying in {wait:.2f}s...')
                    time.sleep(wait)
                    wait *= backoff
        return wrapper
    return decorator


# ── 2. Timing decorator ───────────────────────────────────────────────────────

def timeit_dec(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        t0     = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        print(f'  ⏱ {func.__name__}() took {elapsed*1000:.2f}ms')
        return result
    return wrapper


# ── Usage ─────────────────────────────────────────────────────────────────────
_call_count = 0

@retry(max_attempts=3, delay=0.05, backoff=2, exceptions=(ConnectionError,))
@timeit_dec
def fetch_user_data(user_id: int) -> dict:
    """Simulates a flaky API call that succeeds on the 3rd attempt."""
    global _call_count
    _call_count += 1
    if _call_count < 3:
        raise ConnectionError(f'Network blip #{_call_count}')
    return {'id': user_id, 'name': 'Alice', 'email': 'alice@example.com'}


print("Fetching user data (will retry twice)...")
result = fetch_user_data(42)
print(f"  ✓ Result: {result}")

# ── functools.wraps preserves metadata ───────────────────────────────────────
print(f"\n  Function name preserved: {fetch_user_data.__name__!r}")
print(f"  Docstring preserved:     {fetch_user_data.__doc__!r}")


In [ ]:
# ── Bonus: Class-based decorator (stateful) ────────────────────────────────

class RateLimit:
    """Decorator that limits how often a function can be called."""

    def __init__(self, calls_per_second: float):
        self.min_interval = 1.0 / calls_per_second
        self.last_called  = 0.0

    def __call__(self, func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            now = time.monotonic()
            wait_time = self.min_interval - (now - self.last_called)
            if wait_time > 0:
                time.sleep(wait_time)
            self.last_called = time.monotonic()
            return func(*args, **kwargs)
        return wrapper


@RateLimit(calls_per_second=5)   # max 5 calls per second
def send_notification(msg: str):
    print(f'  📢 Sent: {msg}')


print("Sending 3 notifications (rate limited to 5/sec):")
for i in range(3):
    send_notification(f'Alert #{i+1}')


---
## ⚙️ Topic 5: Generators & Coroutines
`────────────────────────────────────────────────────────────`


### What is it?
**Generators** are functions that `yield` values one at a time, suspending execution
between calls — enabling **lazy evaluation** of arbitrarily large sequences.

**Coroutines** extend generators to support bidirectional data flow via `send()`.

### Why it matters
```python
# ❌ Loads 10GB into RAM first
data = [process(row) for row in read_all_rows()]

# ✅ Processes one row at a time — O(1) memory
data = (process(row) for row in stream_rows())
```

### Real-World Use Cases
> Streaming ETL pipelines · Infinite data streams · Log file processing
> · Database cursor iteration · Cooperative multitasking


In [ ]:
import io, csv
from typing import Iterator, Dict

# ── Simulated large dataset (in-memory for demo) ──────────────────────────────
SAMPLE_CSV = """id,name,price,category
1,Widget A,9.99,tools
2,Gadget B,invalid,electronics
3,Widget C,24.50,tools
4,Gadget D,15.00,electronics
5,,8.75,tools
6,Widget E,33.20,tools
7,Gadget F,12.00,electronics
"""

# ── Generator pipeline stages ─────────────────────────────────────────────────

def read_rows(csv_text: str) -> Iterator[Dict]:
    """Stage 1: yield one dict per CSV row — never loads the whole file."""
    reader = csv.DictReader(io.StringIO(csv_text))
    for row in reader:
        yield row   # suspend here; resume when caller asks for next value


def parse_and_validate(rows: Iterator) -> Iterator[Dict]:
    """Stage 2: type-coerce and skip malformed rows."""
    for row in rows:
        try:
            if not row.get('name', '').strip():
                print(f'  ⚠ Skipping row {row["id"]}: missing name')
                continue
            yield {
                'id':       int(row['id']),
                'name':     row['name'].strip(),
                'price':    float(row['price']),
                'category': row['category'],
            }
        except (ValueError, KeyError) as e:
            print(f'  ⚠ Skipping malformed row {row.get("id","?")}: {e}')


def apply_discount(rows: Iterator, pct: float) -> Iterator[Dict]:
    """Stage 3: transform prices."""
    for row in rows:
        row['price'] = round(row['price'] * (1 - pct), 2)
        row['discounted'] = True
        yield row


def filter_by_category(rows: Iterator, category: str) -> Iterator[Dict]:
    """Stage 4: filter."""
    for row in rows:
        if row['category'] == category:
            yield row


def batch(rows: Iterator, size: int) -> Iterator[list]:
    """Stage 5: accumulate into fixed-size chunks for bulk DB inserts."""
    chunk = []
    for row in rows:
        chunk.append(row)
        if len(chunk) == size:
            yield chunk
            chunk = []
    if chunk:
        yield chunk


# ── Compose the lazy pipeline ─────────────────────────────────────────────────
pipeline = batch(
    filter_by_category(
        apply_discount(
            parse_and_validate(
                read_rows(SAMPLE_CSV)
            ),
            pct=0.10
        ),
        category='tools'
    ),
    size=2
)

print("=== Processing pipeline output ===")
for i, chunk in enumerate(pipeline, 1):
    print(f"  Batch {i}: {chunk}")


In [ ]:
# ── Coroutine with send() — bidirectional data flow ───────────────────────────

def running_average():
    """Coroutine: send numbers in, get running average back."""
    total = 0.0
    count = 0
    value = None
    while True:
        value = yield (total / count if count else 0)   # yield avg, receive next
        total += value
        count += 1


gen = running_average()
next(gen)   # prime the coroutine (advance to first yield)

print("=== Running average coroutine ===")
for n in [10, 20, 30, 40, 50]:
    avg = gen.send(n)
    print(f'  After sending {n:>3}: running average = {avg:.1f}')

gen.close()


---
## ⚡ Topic 6: Asyncio (async / await)
`────────────────────────────────────────────────────────────`


### What is it?
**Asyncio** provides single-threaded concurrency via cooperative multitasking.
Coroutines voluntarily yield control at `await` points, letting the event loop run
other coroutines while waiting for I/O.

### The mental model
```
Thread-based:   Thread1 [=====WAIT=====][===RUN===]
                Thread2        [===RUN===][=====WAIT=====]

Asyncio:        Loop    [==RUN==][await][==RUN==][await][==RUN==]
                         task A          task B          task A
```

### Real-World Use Cases
> FastAPI / aiohttp servers · Web scrapers · Real-time chat · IoT data ingestion
> · Concurrent DB queries with asyncpg


In [ ]:
import asyncio, time, random

# ── Simulated async I/O operations ────────────────────────────────────────────

async def fetch_user(user_id: int) -> dict:
    """Simulate a database or HTTP call with variable latency."""
    delay = random.uniform(0.05, 0.2)
    await asyncio.sleep(delay)        # yield control while 'waiting for I/O'
    return {'id': user_id, 'name': f'User_{user_id}', 'latency_ms': round(delay*1000)}


async def fetch_orders(user_id: int) -> list:
    delay = random.uniform(0.05, 0.15)
    await asyncio.sleep(delay)
    return [{'order_id': user_id * 10 + i} for i in range(3)]


# ── Sequential vs Concurrent comparison ──────────────────────────────────────

async def load_dashboard_sequential(user_ids):
    """Awaits each call one-by-one."""
    results = []
    for uid in user_ids:
        user   = await fetch_user(uid)
        orders = await fetch_orders(uid)
        results.append({'user': user, 'orders': orders})
    return results


async def load_dashboard_concurrent(user_ids):
    """Fires all coroutines at once with gather()."""
    tasks = [
        asyncio.gather(fetch_user(uid), fetch_orders(uid))
        for uid in user_ids
    ]
    pairs = await asyncio.gather(*tasks)
    return [{'user': u, 'orders': o} for u, o in pairs]


# ── Semaphore: limit concurrency ──────────────────────────────────────────────

async def controlled_fetch(sem: asyncio.Semaphore, uid: int) -> dict:
    async with sem:          # blocks when 3 coroutines are already active
        return await fetch_user(uid)


async def main():
    user_ids = list(range(1, 11))   # 10 users

    # Sequential
    t0 = time.monotonic()
    await load_dashboard_sequential(user_ids)
    seq_time = time.monotonic() - t0

    # Concurrent
    t0 = time.monotonic()
    results = await load_dashboard_concurrent(user_ids)
    con_time = time.monotonic() - t0

    print(f"Sequential:  {seq_time:.2f}s")
    print(f"Concurrent:  {con_time:.2f}s  ({seq_time/con_time:.1f}x faster)")
    print(f"\nSample result: {results[0]['user']}")

    # Semaphore demo (max 3 concurrent)
    sem = asyncio.Semaphore(3)
    t0  = time.monotonic()
    sem_results = await asyncio.gather(*[controlled_fetch(sem, uid) for uid in user_ids])
    sem_time = time.monotonic() - t0
    print(f"\nWith Semaphore(3): {sem_time:.2f}s")
    print(f"Fetched {len(sem_results)} users")


asyncio.run(main())


---
## 🧵 Topic 7: Multithreading
`────────────────────────────────────────────────────────────`


### What is it?
Python's `threading` module runs multiple threads in the **same process and memory space**.
Due to the GIL, only one thread executes Python bytecode at a time — but threads overlap
on I/O waits, making them useful for **I/O-bound parallelism**.

### Real-World Use Cases
> Background workers in web apps · Parallel file downloads · GUI responsiveness
> · Database connection pools · Concurrent API polling

### Thread safety tools
| Tool | Purpose |
|------|---------|
| `threading.Lock` | Mutual exclusion |
| `threading.Event` | Signal between threads |
| `queue.Queue` | Thread-safe data sharing |
| `concurrent.futures.ThreadPoolExecutor` | High-level pool |


In [ ]:
import threading, queue, time, random
from concurrent.futures import ThreadPoolExecutor, as_completed

# ── Thread-safe counter with Lock ─────────────────────────────────────────────

class SafeCounter:
    def __init__(self):
        self.value = 0
        self._lock = threading.Lock()

    def increment(self, n=1):
        with self._lock:            # only one thread at a time
            self.value += n


counter = SafeCounter()
threads = [threading.Thread(target=counter.increment, args=(1,)) for _ in range(1000)]
for t in threads: t.start()
for t in threads: t.join()
print(f"Counter (should be 1000): {counter.value}")


# ── Producer / Consumer with Queue ────────────────────────────────────────────

def producer(q: queue.Queue, items: list):
    for item in items:
        time.sleep(random.uniform(0.01, 0.05))   # simulate data arrival
        q.put(item)
        print(f'  📥 Produced: {item}')
    q.put(None)   # sentinel


def consumer(q: queue.Queue, results: list):
    while True:
        item = q.get()
        if item is None:
            break
        time.sleep(random.uniform(0.01, 0.03))   # simulate processing
        results.append(item * 2)
        print(f'  📤 Consumed: {item} → {item*2}')
        q.task_done()


data    = [10, 20, 30, 40, 50]
q       = queue.Queue(maxsize=3)   # buffer of 3
results = []

print("\n=== Producer/Consumer demo ===")
p = threading.Thread(target=producer, args=(q, data))
c = threading.Thread(target=consumer, args=(q, results))
p.start(); c.start()
p.join();  c.join()
print(f"Results: {sorted(results)}")


# ── ThreadPoolExecutor for parallel I/O tasks ─────────────────────────────────

def simulate_api_call(endpoint: str) -> dict:
    time.sleep(random.uniform(0.05, 0.2))
    return {'endpoint': endpoint, 'status': 200, 'data': f'response_from_{endpoint}'}


endpoints = ['/users', '/products', '/orders', '/analytics', '/health']

print("\n=== ThreadPoolExecutor (parallel API calls) ===")
t0 = time.monotonic()
with ThreadPoolExecutor(max_workers=5) as pool:
    futures = {pool.submit(simulate_api_call, ep): ep for ep in endpoints}
    for future in as_completed(futures):
        result = future.result()
        print(f'  ✓ {result["endpoint"]} → {result["data"]}')
print(f"All calls completed in {time.monotonic()-t0:.2f}s (would be ~0.75s sequentially)")


---
## 🖥️ Topic 8: Multiprocessing
`────────────────────────────────────────────────────────────`


### What is it?
`multiprocessing` spawns separate OS processes, each with its **own Python interpreter
and GIL**. True CPU parallelism is achieved because multiple cores run simultaneously.

### GIL-free parallelism
```
Threading (GIL):          Process 1: [run][WAIT][run][WAIT]
                          Process 2:         [WAIT][run][WAIT]  ← blocked by GIL

Multiprocessing (no GIL): Process 1: [run][run][run][run]  ← full core
                           Process 2: [run][run][run][run]  ← full core
```

### Real-World Use Cases
> ML training data preprocessing · Image/video batch processing
> · Scientific computation · Parallel test suites · CPU-intensive APIs


In [ ]:
import multiprocessing, time, math
from concurrent.futures import ProcessPoolExecutor

# ── CPU-intensive task ────────────────────────────────────────────────────────

def is_prime(n: int) -> bool:
    """CPU-bound: check if n is prime."""
    if n < 2: return False
    if n < 4: return True
    if n % 2 == 0 or n % 3 == 0: return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True


def count_primes_in_range(args):
    """Count primes between start and end — runs in a worker process."""
    start, end = args
    return sum(1 for n in range(start, end) if is_prime(n))


# ── Sequential baseline ────────────────────────────────────────────────────────
N = 500_000
print(f"Counting primes up to {N:,} ...")

t0 = time.monotonic()
seq_count = count_primes_in_range((2, N))
seq_time  = time.monotonic() - t0
print(f"  Sequential: {seq_count:,} primes in {seq_time:.2f}s")


# ── Parallel with ProcessPoolExecutor ─────────────────────────────────────────
cpu_count = multiprocessing.cpu_count()
chunk     = N // cpu_count
ranges    = [(i * chunk, (i + 1) * chunk) for i in range(cpu_count)]
# make sure we cover the full range
ranges[-1] = (ranges[-1][0], N)

t0 = time.monotonic()
with ProcessPoolExecutor(max_workers=cpu_count) as pool:
    counts   = list(pool.map(count_primes_in_range, ranges))
par_count = sum(counts)
par_time  = time.monotonic() - t0

print(f"  Parallel ({cpu_count} cores): {par_count:,} primes in {par_time:.2f}s")
print(f"  Speedup: {seq_time/par_time:.1f}x")
assert seq_count == par_count, "Results must match!"
print("  ✓ Results match")


In [ ]:
# ── Shared memory between processes ──────────────────────────────────────────

from multiprocessing import Value, Array, Process

def worker_shared(counter, lock):
    for _ in range(100):
        with lock:
            counter.value += 1


shared_counter = Value('i', 0)   # shared integer
lock           = multiprocessing.Lock()

procs = [Process(target=worker_shared, args=(shared_counter, lock)) for _ in range(5)]
for p in procs: p.start()
for p in procs: p.join()
print(f"Shared counter (should be 500): {shared_counter.value}")


---
## 🔐 Topic 9: Global Interpreter Lock (GIL)
`────────────────────────────────────────────────────────────`


### What is it?
The **GIL** is a mutex in CPython that allows only **one thread to execute Python
bytecode at a time**. It protects Python's internal reference-counting from race
conditions — but prevents true multi-threaded CPU parallelism.

### Key facts
- **Released** during C-level I/O (socket, file reads) → threads overlap on I/O ✅
- **Released** by NumPy/SciPy during C loops → threads can parallelize ✅
- **Held** during pure Python bytecode execution → threads queue up ❌
- **Python 3.13+** introduces a free-threaded (no-GIL) build option 🔜

### Impact on your code
| Workload | Threading | Multiprocessing |
|----------|-----------|-----------------|
| I/O-bound (network, disk) | ✅ Works great | Overkill |
| CPU-bound (pure Python) | ❌ No speedup | ✅ Use this |
| CPU-bound (NumPy/C ext.) | ✅ Works | ✅ Also works |


In [ ]:
import threading, multiprocessing, time

def cpu_burn(n: int):
    """Pure CPU work — counts down. GIL is held throughout."""
    while n > 0:
        n -= 1

N = 10_000_000   # reduced for notebook speed

# ── 1. Single thread (baseline) ───────────────────────────────────────────────
t0 = time.monotonic()
cpu_burn(N)
single_time = time.monotonic() - t0
print(f"Single thread:   {single_time:.3f}s")

# ── 2. Two threads — GIL prevents true parallelism ───────────────────────────
t0 = time.monotonic()
t1 = threading.Thread(target=cpu_burn, args=(N // 2,))
t2 = threading.Thread(target=cpu_burn, args=(N // 2,))
t1.start(); t2.start()
t1.join();  t2.join()
thread_time = time.monotonic() - t0
print(f"Two threads:     {thread_time:.3f}s  ← ~same or SLOWER (GIL overhead)")

# ── 3. Two processes — real parallelism ──────────────────────────────────────
t0 = time.monotonic()
p1 = multiprocessing.Process(target=cpu_burn, args=(N // 2,))
p2 = multiprocessing.Process(target=cpu_burn, args=(N // 2,))
p1.start(); p2.start()
p1.join();  p2.join()
proc_time = time.monotonic() - t0
print(f"Two processes:   {proc_time:.3f}s  ← ~2x speedup on 2 cores")

ratio = single_time / proc_time
print(f"\nConclusion: Processes gave {ratio:.1f}x speedup; threads gave"
      f" {single_time/thread_time:.1f}x (GIL is real!)")


---
## ♻️ Topic 10: Memory Management & Garbage Collection
`────────────────────────────────────────────────────────────`


### What is it?
CPython uses **reference counting** as its primary strategy — every object tracks
how many references point to it and is freed *immediately* when the count reaches zero.

A **cyclic garbage collector** (in the `gc` module) handles reference cycles that
reference counting alone cannot break.

### Three generations
Objects are promoted through 3 generations; long-lived objects are collected less often:
```
Gen 0 (new)     threshold=700  ← collected most frequently
Gen 1 (survived 1 collection)  threshold=10
Gen 2 (long-lived)             threshold=10  ← collected rarely
```


In [ ]:
import gc, sys, tracemalloc

# ── Reference counting ────────────────────────────────────────────────────────
x = []
print(f"Refs to x after creation:      {sys.getrefcount(x)}")  # 2: x + getrefcount arg

y = x
print(f"Refs to x after y=x:           {sys.getrefcount(x)}")  # 3

del y
print(f"Refs to x after del y:         {sys.getrefcount(x)}")  # 2

# ── Reference cycle — memory leak without cyclic GC ──────────────────────────
class Node:
    def __init__(self, name):
        self.name  = name
        self.child = None
    def __del__(self):
        pass  # called when freed

gc.disable()   # pause cyclic collector so we can observe the leak
gc.collect()

before = len(gc.get_objects())

a = Node('A')
b = Node('B')
a.child = b    # A → B
b.child = a    # B → A  ← cycle!

del a, b       # refcount never hits 0 — MEMORY LEAK without cyclic GC

leaked = len(gc.get_objects()) - before
print(f"\nObjects leaked (cycle, GC disabled): {leaked}")

gc.enable()
collected = gc.collect()   # cyclic GC finds and frees the cycle
print(f"gc.collect() freed: {collected} objects")

# ── tracemalloc — find what's using memory ────────────────────────────────────
tracemalloc.start()

big_list = [bytearray(1024) for _ in range(1000)]   # 1 MB

snapshot = tracemalloc.take_snapshot()
top = snapshot.statistics('lineno')
print("\nTop memory consumers:")
for stat in top[:3]:
    print(f"  {stat}")

del big_list
tracemalloc.stop()

# ── Tuning GC thresholds for high-throughput servers ─────────────────────────
print(f"\nDefault thresholds: {gc.get_threshold()}")
gc.set_threshold(1000, 20, 20)   # fewer collections = lower pause frequency
print(f"Tuned thresholds:   {gc.get_threshold()}")


---
## 🔗 Topic 11: Weak References
`────────────────────────────────────────────────────────────`


### What is it?
A **weak reference** lets you hold a reference to an object **without incrementing
its reference count**. If no strong references remain, the object is garbage collected
and the weak reference becomes `None`.

### Strong vs Weak
```python
strong = obj   # refcount += 1  — obj won't be GC'd while this exists
weak   = weakref.ref(obj)   # refcount unchanged — obj may disappear
```

### Real-World Use Cases
> LRU caches where entries should vanish when unused
> · Observer/event systems where listeners shouldn't own the source
> · Object registries in plugin architectures


In [ ]:
import weakref, gc

class ExpensiveResource:
    def __init__(self, key: str):
        self.key  = key
        self.data = list(range(10_000))   # simulate expensive object
        print(f'  [+] Created {key!r}')

    def __del__(self):
        print(f'  [-] Freed   {key!r}' if False else f'  [-] Freed   {self.key!r}')

    def compute(self):
        return sum(self.data)


# ── WeakValueDictionary: cache that releases entries automatically ─────────────
class SmartCache:
    def __init__(self):
        self._store = weakref.WeakValueDictionary()

    def get_or_create(self, key: str) -> ExpensiveResource:
        obj = self._store.get(key)
        if obj is None:
            print(f'  MISS — creating {key!r}')
            obj = ExpensiveResource(key)
            self._store[key] = obj
        else:
            print(f'  HIT  — reusing {key!r}')
        return obj


cache = SmartCache()

print("=== First access — creates objects ===")
r1 = cache.get_or_create('model_weights')
r2 = cache.get_or_create('embeddings')

print("\n=== Second access — cache hits ===")
r1b = cache.get_or_create('model_weights')
print(f'Same object? {r1 is r1b}')  # True

print("\n=== Releasing strong references ===")
del r1, r1b, r2
gc.collect()

print("\n=== Re-access after GC — cache misses again ===")
r3 = cache.get_or_create('model_weights')   # recreated
del r3

# ── weakref.ref — low-level callback on death ─────────────────────────────────
print("\n=== weakref.ref with finalizer callback ===")
obj = ExpensiveResource('singleton')

def on_death(ref):
    print(f'  💀 Weak ref notified: object is gone')

weak = weakref.ref(obj, on_death)
print(f'Alive: {weak()}')
del obj
gc.collect()
print(f'Dead:  {weak()}')


---
## 🗂️ Topic 12: __slots__
`────────────────────────────────────────────────────────────`


### What is it?
By default, Python stores instance attributes in a per-instance `__dict__` (a hash map).
Defining `__slots__` replaces this with a compact, fixed-size array of slots —
reducing per-instance memory by **40–60%** and speeding up attribute access.

### Without vs With
```python
class Normal:               # each instance has a __dict__ (hash map overhead)
    def __init__(self): self.x = 1

class Slotted:              # each instance has fixed-size C-level slots
    __slots__ = ('x',)
    def __init__(self): self.x = 1
```

### Real-World Use Cases
> Financial tick data (millions of events/second) · ML feature vectors
> · Network packet objects · Any class instantiated millions of times


In [ ]:
import sys, timeit

# ── Memory comparison ─────────────────────────────────────────────────────────

class TradeEvent:
    """Without __slots__ — each instance carries a __dict__."""
    def __init__(self, symbol, price, qty, ts):
        self.symbol = symbol
        self.price  = price
        self.qty    = qty
        self.ts     = ts


class TradeEventSlotted:
    """With __slots__ — no per-instance __dict__."""
    __slots__ = ('symbol', 'price', 'qty', 'ts')

    def __init__(self, symbol, price, qty, ts):
        self.symbol = symbol
        self.price  = price
        self.qty    = qty
        self.ts     = ts


t1 = TradeEvent('AAPL', 182.35, 100, 1_700_000_000)
t2 = TradeEventSlotted('AAPL', 182.35, 100, 1_700_000_000)

size_normal  = sys.getsizeof(t1) + sys.getsizeof(t1.__dict__)
size_slotted = sys.getsizeof(t2)

print(f"Without __slots__: {size_normal} bytes")
print(f"With    __slots__: {size_slotted} bytes")
print(f"Reduction:         {(1 - size_slotted/size_normal)*100:.0f}%")

# ── Speed comparison ──────────────────────────────────────────────────────────
setup = "from __main__ import TradeEvent, TradeEventSlotted; t1=TradeEvent('AAPL',1,1,1); t2=TradeEventSlotted('AAPL',1,1,1)"
t_normal  = timeit.timeit("t1.price = 183.0; _ = t1.price", setup=setup, number=1_000_000)
t_slotted = timeit.timeit("t2.price = 183.0; _ = t2.price", setup=setup, number=1_000_000)
print(f"\nAttribute access (1M iterations):")
print(f"  Normal:  {t_normal:.3f}s")
print(f"  Slotted: {t_slotted:.3f}s  ({t_normal/t_slotted:.2f}x faster)")

# ── Constraint: cannot add arbitrary attributes ───────────────────────────────
try:
    t2.volume = 5000   # not in __slots__!
except AttributeError as e:
    print(f"\n✗ {e}")

# ── hasattr checks ─────────────────────────────────────────────────────────────
print(f"\nhas __dict__: normal={hasattr(t1,'__dict__')}, slotted={hasattr(t2,'__dict__')}")
print(f"has __slots__: normal={hasattr(t1,'__slots__')}, slotted={hasattr(t2,'__slots__')}")


---
## 📋 Topic 13: Abstract Base Classes (ABC)
`────────────────────────────────────────────────────────────`


### What is it?
ABCs define **interfaces** — classes with abstract methods that subclasses *must* implement.
Python raises `TypeError` when you try to instantiate a class with unimplemented abstract methods.

### Why use ABCs over duck typing?
- Catch missing method implementations **at instantiation time**, not at call time
- Enable meaningful `isinstance()` checks across plugin implementations
- Serve as documentation of the required interface

### Real-World Use Cases
> Payment gateway adapters · Storage backends (S3, GCS, local) · Notification channels
> · Plugin systems · serialisation codecs


In [ ]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
from decimal import Decimal

@dataclass
class PaymentResult:
    success:        bool
    transaction_id: str
    message:        str


class PaymentGateway(ABC):
    """Interface contract: all payment backends must implement these methods."""

    @abstractmethod
    def charge(self, amount: Decimal, card_token: str) -> PaymentResult:
        """Charge the card."""
        ...

    @abstractmethod
    def refund(self, transaction_id: str, amount: Decimal) -> PaymentResult:
        """Issue a refund."""
        ...

    @abstractmethod
    def verify_card(self, card_token: str) -> bool:
        """Check if the card is valid."""
        ...

    def charge_if_valid(self, amount: Decimal, card_token: str) -> PaymentResult:
        """Template method — calls abstract methods."""
        if not self.verify_card(card_token):
            return PaymentResult(False, '', 'Card verification failed')
        return self.charge(amount, card_token)


# ── Concrete implementations ──────────────────────────────────────────────────

class StripeGateway(PaymentGateway):
    def charge(self, amount, card_token):
        return PaymentResult(True, 'ch_stripe_123', f'Stripe charged ${amount}')
    def refund(self, transaction_id, amount):
        return PaymentResult(True, 're_stripe_456', f'Stripe refunded ${amount}')
    def verify_card(self, card_token):
        return card_token.startswith('tok_')


class PayPalGateway(PaymentGateway):
    def charge(self, amount, card_token):
        return PaymentResult(True, 'pp_789', f'PayPal charged ${amount}')
    def refund(self, transaction_id, amount):
        return PaymentResult(True, 'pp_ref_012', f'PayPal refunded ${amount}')
    def verify_card(self, card_token):
        return True


# ── Cannot instantiate abstract class ────────────────────────────────────────
try:
    PaymentGateway()
except TypeError as e:
    print(f"✗ {e}")

# ── Cannot use incomplete implementation ────────────────────────────────────
class IncompleteGateway(PaymentGateway):
    def charge(self, amount, card_token):
        return PaymentResult(True, 'x', 'ok')
    # forgot refund and verify_card!

try:
    IncompleteGateway()
except TypeError as e:
    print(f"✗ {e}")

# ── Correct usage ────────────────────────────────────────────────────────────
def process_order(gateway: PaymentGateway, amount, token):
    result = gateway.charge_if_valid(amount, token)
    print(f"  {'✓' if result.success else '✗'} {result.message}")
    return result

print("\n=== Processing orders ===")
process_order(StripeGateway(),  Decimal('49.99'), 'tok_visa_xxxx')
process_order(PayPalGateway(),  Decimal('29.99'), 'pp_email_abc')
process_order(StripeGateway(),  Decimal('19.99'), 'invalid_token')


---
## 🐒 Topic 14: Monkey Patching
`────────────────────────────────────────────────────────────`


### What is it?
**Monkey patching** replaces or augments attributes of a module or class *at runtime*.
It's widely used in testing (mocking external services), adding observability to
third-party code, and applying emergency fixes without forking a library.

### Real-World Use Cases
> `unittest.mock.patch` in tests · Gevent's cooperative I/O patches
> · Adding timing/logging to third-party code · Hot-fixing library bugs


In [ ]:
import time, functools
from unittest.mock import patch, MagicMock

# ── 1. Basic monkey patch ─────────────────────────────────────────────────────

class EmailService:
    def send(self, to: str, subject: str, body: str) -> bool:
        # In real code this would hit an SMTP server
        raise ConnectionError("No SMTP server available in notebook!")


email_service = EmailService()

# Save original, replace with mock
_original_send = EmailService.send
EmailService.send = lambda self, *a, **kw: True   # always succeeds

result = email_service.send('alice@example.com', 'Hello', 'Body')
print(f"Patched send returned: {result}")

EmailService.send = _original_send   # restore


# ── 2. unittest.mock.patch as context manager ────────────────────────────────

def send_welcome_email(service, user_email: str) -> bool:
    """Function under test — depends on EmailService.send."""
    return service.send(user_email, 'Welcome!', 'Thanks for signing up.')


print("\n=== Unit test with mock.patch ===")
with patch.object(EmailService, 'send', return_value=True) as mock_send:
    result = send_welcome_email(email_service, 'bob@example.com')
    mock_send.assert_called_once_with('bob@example.com', 'Welcome!', 'Thanks for signing up.')
    print(f"  ✓ Email sent: {result}")
    print(f"  ✓ Called with: {mock_send.call_args}")


# ── 3. Runtime instrumentation — add timing to any function ──────────────────

def instrument(module, func_name: str):
    """Wraps module.func_name with a timing layer without touching source code."""
    original = getattr(module, func_name)

    @functools.wraps(original)
    def timed(*args, **kwargs):
        t0  = time.perf_counter()
        res = original(*args, **kwargs)
        ms  = (time.perf_counter() - t0) * 1000
        print(f'  [TIMING] {module.__name__}.{func_name}() → {ms:.3f}ms')
        return res

    setattr(module, func_name, timed)
    return original   # return original so we can restore later


import json
original_dumps = instrument(json, 'dumps')

json.dumps({'hello': 'world', 'numbers': list(range(100))})
json.dumps([1, 2, 3])

json.dumps = original_dumps   # restore
print("  Restored json.dumps")


---
## 📦 Topic 15: Function Closures
`────────────────────────────────────────────────────────────`


### What is it?
A **closure** is a function that *captures variables from its enclosing scope*
even after that scope has exited. Closures are the mechanism behind decorators,
factory functions, and stateful callbacks.

```python
def outer():
    x = 10              # ← captured by inner
    def inner():
        return x        # ← accesses x from enclosing scope
    return inner        # outer() has exited, but x lives on in inner's closure

f = outer()
f()   # → 10
```

### Real-World Use Cases
> Currency/unit formatters · Middleware pipelines · Event callbacks
> · Partial application · Memoization helpers


In [ ]:
# ── 1. Factory functions ───────────────────────────────────────────────────────

def make_formatter(prefix: str, suffix: str, decimals: int = 2):
    """Returns a specialised formatting function closed over the parameters."""
    def formatter(value: float) -> str:
        return f'{prefix}{value:,.{decimals}f}{suffix}'
    return formatter


usd  = make_formatter('$', '')
eur  = make_formatter('', ' €')
gbp  = make_formatter('£', '')
perc = make_formatter('', '%', decimals=1)

for fn, val in [(usd, 1_234.56), (eur, 1_234.56), (gbp, 999.0), (perc, 12.345)]:
    print(f'  {fn(val)}')


# ── 2. Inspect the closure ─────────────────────────────────────────────────────
import inspect

print(f"\nclosure vars: {[c.cell_contents for c in usd.__code__.co_freevars and usd.__closure__]}")
print(f"free vars:    {usd.__code__.co_freevars}")


# ── 3. Stateful closure (accumulator) ─────────────────────────────────────────

def make_accumulator(initial=0):
    total = [initial]   # list cell is mutable — avoids need for 'nonlocal' in Py2

    def add(value):
        total[0] += value
        return total[0]

    def reset():
        total[0] = initial

    add.reset = reset   # attach helper to the closure function
    return add


sales = make_accumulator()
print(f"\n  Sales: {sales(100)}")
print(f"  Sales: {sales(250)}")
print(f"  Sales: {sales(75)}")
sales.reset()
print(f"  After reset: {sales(50)}")


# ── 4. Closure gotcha — late binding ─────────────────────────────────────────
print("\n=== Late binding gotcha ===")
funcs_bad = [lambda: i for i in range(5)]      # all capture the SAME i
funcs_ok  = [lambda i=i: i for i in range(5)]  # default arg captures current i

print(f"  Bad (late binding):  {[f() for f in funcs_bad]}")   # [4,4,4,4,4]
print(f"  Fixed (default arg): {[f() for f in funcs_ok]}")    # [0,1,2,3,4]


---
## 🔍 Topic 16: Introspection & Reflection
`────────────────────────────────────────────────────────────`


### What is it?
**Introspection** — examining types, attributes, and call signatures at runtime.
**Reflection** — modifying behaviour based on that introspection.

Python exposes these via: `inspect`, `dir`, `type`, `vars`, `getattr`, `setattr`,
`hasattr`, `isinstance`, `issubclass`.

### Real-World Use Cases
> Serialisation frameworks (pydantic, marshmallow) · Dependency injection containers
> · CLI generators (Fire, Typer) · API documentation generators (FastAPI auto-docs)
> · Test frameworks (pytest fixture discovery)


In [ ]:
import inspect

# ── Explore a class at runtime ─────────────────────────────────────────────────

class DataProcessor:
    """Processes data files with various transformation options."""

    MAX_BATCH = 1000

    def load(self, filepath: str, encoding: str = 'utf-8') -> int:
        """Load data from a file. Returns row count."""
        return 42

    def transform(self, operation: str, column: str, inplace: bool = False) -> bool:
        """Apply a named operation to a column."""
        return True

    def export(self, output: str, fmt: str = 'csv', compress: bool = False) -> str:
        """Export data to a file. Returns output path."""
        return output

    def _internal(self):
        pass  # private


proc = DataProcessor()

# ── What methods does this object have? ───────────────────────────────────────
public_methods = {
    name: func
    for name, func in inspect.getmembers(proc, predicate=inspect.ismethod)
    if not name.startswith('_')
}
print("Public methods:", list(public_methods))

# ── Inspect method signatures ─────────────────────────────────────────────────
for name, method in public_methods.items():
    sig = inspect.signature(method)
    print(f"\n  {name}{sig}")
    for pname, param in sig.parameters.items():
        ann     = param.annotation if param.annotation != inspect.Parameter.empty else 'any'
        default = f' = {param.default!r}' if param.default != inspect.Parameter.empty else ''
        print(f'    ├ {pname}: {ann}{default}')

# ── Auto-generate CLI dispatcher ──────────────────────────────────────────────
def auto_dispatch(obj, command: str, *cli_args):
    methods = {n: m for n, m in inspect.getmembers(obj, inspect.ismethod)
               if not n.startswith('_')}
    if command not in methods:
        print(f"  Unknown: {command}. Available: {list(methods)}")
        return
    method = methods[command]
    sig    = inspect.signature(method)
    params = list(sig.parameters.values())
    coerced = []
    for i, param in enumerate(params):
        if i >= len(cli_args):
            coerced.append(param.default)
        else:
            ann = param.annotation
            coerced.append(ann(cli_args[i]) if ann != inspect.Parameter.empty else cli_args[i])
    result = method(*coerced)
    print(f"  [{command}] → {result!r}")

print("\n=== Auto CLI dispatcher ===")
auto_dispatch(proc, 'load',      'sales.csv', 'latin-1')
auto_dispatch(proc, 'transform', 'uppercase', 'name', 'True')
auto_dispatch(proc, 'export',    'out.csv')
auto_dispatch(proc, 'missing_command')


---
## 🎛️ Topic 17: Dynamic Attribute Access (__getattr__, __getattribute__)
`────────────────────────────────────────────────────────────`


### What is it?
| Method | When called | Risk |
|--------|-------------|------|
| `__getattr__` | Only when *normal lookup fails* | Low — safe default |
| `__getattribute__` | On *every* attribute access | High — infinite recursion if careless |

### Real-World Use Cases
> ORM lazy-loading (accessing `row.author` triggers a DB query)
> · Remote object proxies · Config objects that read env vars on demand
> · Attribute-driven query builders (SQLAlchemy, Django ORM `filter()`)


In [ ]:
# ── Lazy database row proxy ────────────────────────────────────────────────────

class FakeDB:
    """Simulates a database with a find(table, pk) method."""
    _tables = {
        'users':      {1: {'id': 1, 'name': 'Alice', 'role': 'admin'},
                       2: {'id': 2, 'name': 'Bob',   'role': 'user'}},
        'categories': {5: {'id': 5, 'name': 'Python'},
                       6: {'id': 6, 'name': 'Django'}},
    }
    _query_count = 0

    def find(self, table: str, pk: int) -> dict:
        self._query_count += 1
        print(f'  [DB query #{self._query_count}] SELECT * FROM {table} WHERE id={pk}')
        return self._tables.get(table, {}).get(pk)


class LazyRow:
    """
    Wraps a DB row dict and lazily fetches related rows on first access.
    Caches results to avoid redundant queries.
    """
    _RELATIONS = {'author': 'users', 'category': 'categories'}

    def __init__(self, data: dict, db: FakeDB):
        object.__setattr__(self, '_data',  data)
        object.__setattr__(self, '_db',    db)
        object.__setattr__(self, '_cache', {})

    def __getattr__(self, name: str):
        # Only called when normal lookup fails
        data  = object.__getattribute__(self, '_data')
        cache = object.__getattribute__(self, '_cache')
        db    = object.__getattribute__(self, '_db')

        if name in data:                        # direct column
            return data[name]

        rel_table = self._RELATIONS.get(name)   # try relation
        if rel_table:
            fk = data.get(name + '_id')
            if fk is not None:
                if name not in cache:
                    cache[name] = db.find(rel_table, fk)
                return cache[name]

        raise AttributeError(f'No attribute {name!r}')

    def __repr__(self):
        return f"LazyRow({object.__getattribute__(self, '_data')})"


db  = FakeDB()
row = LazyRow({
    'id': 10, 'title': 'Getting Started', 'views': 1337,
    'author_id': 1, 'category_id': 5
}, db)

print("=== Accessing plain columns (no DB query) ===")
print(f"  id:    {row.id}")
print(f"  title: {row.title}")
print(f"  views: {row.views}")
print(f"  DB queries so far: {db._query_count}")

print("\n=== First access to relation — triggers DB query ===")
print(f"  author:   {row.author}")
print(f"  category: {row.category}")

print("\n=== Second access — served from cache ===")
print(f"  author:   {row.author}")
print(f"  category: {row.category}")
print(f"  Total DB queries: {db._query_count}  (only 2, not 4!)")


---
## ⚙️ Topic 18: Data Model & Dunder Methods
`────────────────────────────────────────────────────────────`


### What is it?
Python's **data model** is the set of special (dunder) methods that make user-defined
objects participate in language syntax — arithmetic operators, comparisons, iteration,
hashing, string representation, and more.

### Common dunder methods
| Category | Methods |
|----------|---------|
| Representation | `__repr__`, `__str__`, `__format__` |
| Arithmetic | `__add__`, `__sub__`, `__mul__`, `__truediv__`, `__neg__` |
| Comparison | `__eq__`, `__lt__`, `__le__`, `__gt__`, `__ge__` |
| Container | `__len__`, `__getitem__`, `__setitem__`, `__iter__`, `__contains__` |
| Context | `__enter__`, `__exit__` |
| Lifecycle | `__init__`, `__new__`, `__del__` |
| Hashing | `__hash__`, `__bool__` |


In [ ]:
from __future__ import annotations
from decimal import Decimal
from functools import total_ordering

@total_ordering   # auto-generates <=, >, >= from __eq__ + __lt__
class Money:
    """Immutable monetary value with currency safety."""

    def __init__(self, amount, currency: str = 'USD'):
        self.amount   = Decimal(str(amount))
        self.currency = currency.upper()

    # ── Representation ─────────────────────────────────────────────────────
    def __repr__(self)  -> str: return f"Money({self.amount!r}, {self.currency!r})"
    def __str__(self)   -> str: return f"{self.currency} {self.amount:,.2f}"
    def __format__(self, spec):return format(str(self), spec)

    # ── Equality & Comparison ──────────────────────────────────────────────
    def _assert_same_currency(self, other):
        if not isinstance(other, Money) or other.currency != self.currency:
            raise TypeError(f'Cannot mix {self.currency} and {getattr(other,"currency","??")}')

    def __eq__(self, other) -> bool:
        if not isinstance(other, Money): return NotImplemented
        self._assert_same_currency(other)
        return self.amount == other.amount

    def __lt__(self, other) -> bool:
        self._assert_same_currency(other)
        return self.amount < other.amount

    # ── Arithmetic ─────────────────────────────────────────────────────────
    def __add__(self, other: Money) -> Money:
        self._assert_same_currency(other)
        return Money(self.amount + other.amount, self.currency)

    def __sub__(self, other: Money) -> Money:
        self._assert_same_currency(other)
        return Money(self.amount - other.amount, self.currency)

    def __mul__(self, factor) -> Money:
        return Money(self.amount * Decimal(str(factor)), self.currency)

    def __rmul__(self, factor) -> Money: return self.__mul__(factor)
    def __neg__(self)          -> Money: return Money(-self.amount, self.currency)
    def __abs__(self)          -> Money: return Money(abs(self.amount), self.currency)

    # ── Container / truth ──────────────────────────────────────────────────
    def __bool__(self) -> bool: return self.amount != 0
    def __hash__(self):         return hash((self.amount, self.currency))


# ── Usage demo ────────────────────────────────────────────────────────────────
price    = Money('19.99')
tax      = Money('1.60')
shipping = Money('4.99')

print(f"price:    {price}")
print(f"tax:      {tax}")
print(f"total:    {price + tax + shipping}")
print(f"x3:       {(price + tax + shipping) * 3}")
print(f"discount: {price - Money('5.00')}")

prices = [Money('25'), Money('10'), Money('50'), Money('5'), Money('30')]
print(f"\nsorted:   {sorted(prices)}")
print(f"min:      {min(prices)}")
print(f"max:      {max(prices)}")
print(f"sum:      {sum(prices, Money('0'))}")

print(f"\nbool(Money('0')): {bool(Money('0'))}")
print(f"bool(Money('1')): {bool(Money('1'))}")

try:
    Money('10', 'USD') + Money('10', 'EUR')
except TypeError as e:
    print(f"\n✗ {e}")


---
## ⚡ Topic 19: C Extensions / CPython API
`────────────────────────────────────────────────────────────`


### What is it?
C extensions are compiled shared libraries (`.so`/`.pyd`) that CPython loads directly.
They can bypass the GIL for pure C computation and run **10–100× faster** than Python.

### Authoring approaches
| Tool | Best for | Difficulty |
|------|----------|------------|
| **Cython** | Typed Python-like syntax → C | ★★☆ |
| **pybind11** | C++ integration | ★★★ |
| **CFFI** | Calling existing C libraries | ★★☆ |
| **ctypes** | Calling C libs (stdlib) | ★☆☆ |
| **CPython C API** | Full control | ★★★ |

### Real-World Use Cases
> NumPy · Pandas · SciPy · OpenCV · PyTorch · Cryptography · lxml


In [ ]:
# ── ctypes — calling C stdlib functions (no compilation needed) ───────────────
import ctypes, ctypes.util, timeit, math

# Load the C math library
libm_path = ctypes.util.find_library('m')
if libm_path:
    libm = ctypes.CDLL(libm_path)
    libm.sqrt.restype  = ctypes.c_double
    libm.sqrt.argtypes = [ctypes.c_double]

    # Compare Python math.sqrt vs C sqrt via ctypes
    N = 500_000
    t_py = timeit.timeit(lambda: math.sqrt(12345.6789), number=N)
    t_c  = timeit.timeit(lambda: libm.sqrt(12345.6789), number=N)
    print(f"math.sqrt (Python): {t_py*1000:.1f}ms for {N} calls")
    print(f"libm.sqrt (C):      {t_c*1000:.1f}ms  for {N} calls")
    print(f"Note: ctypes overhead can make C calls slower for trivial ops!")
    print(f"Results match: {abs(math.sqrt(12345.6789) - libm.sqrt(12345.6789)) < 1e-10}")
else:
    print("libm not found on this system")

# ── Simulate what a Cython speedup would look like ────────────────────────────

def mean_python(values: list) -> float:
    """Pure Python mean — bytecode loop."""
    n = len(values)
    if n == 0: return 0.0
    total = 0.0
    for v in values:
        total += v
    return total / n


def mean_builtin(values: list) -> float:
    """Using C-level builtins — effectively a C extension internally."""
    return sum(values) / len(values)


data = list(range(100_000))

t_py  = timeit.timeit(lambda: mean_python(data),  number=20)
t_c   = timeit.timeit(lambda: mean_builtin(data), number=20)
print(f"\nmean_python:  {t_py:.3f}s")
print(f"mean_builtin: {t_c:.3f}s  ({t_py/t_c:.1f}x faster — built-in sum() is a C extension!)")

# ── Cython source (shown as text — requires build step) ───────────────────────
cython_source = '''
# fast_stats.pyx — compile with: python setup.py build_ext --inplace

def mean_cython(list values):
    cdef int    n     = len(values)
    if n == 0: return 0.0
    cdef double total = 0.0
    cdef double v
    for v in values:
        total += v
    return total / n
'''
print("\n--- Cython source (requires compilation) ---")
print(cython_source)


---
## 🖥️ Topic 20: Bytecode & Python VM
`────────────────────────────────────────────────────────────`


### What is it?
Python source is compiled to **bytecode** — a platform-independent instruction set
executed by the CPython virtual machine (`ceval.c`).

```
source.py ──► compile ──► __pycache__/source.cpython-3xx.pyc ──► CPython VM executes
```

### Why it matters
- Explains performance differences between code patterns
- Local variables are array-indexed (fast); globals require dictionary lookup (slow)
- List comprehensions generate bytecode without `STORE_NAME` overhead of explicit loops
- Tools like `dis`, `py_compile`, Nuitka, PyInstaller, py2exe all work at this level


In [ ]:
import dis, timeit

# ── Disassemble a function ────────────────────────────────────────────────────
def add_numbers(a, b):
    result = a + b
    return result

print("=== Bytecode for add_numbers ===")
dis.dis(add_numbers)

# ── Compare bytecode: loop vs list comprehension ─────────────────────────────
def squares_loop(n):
    result = []
    for i in range(n):
        result.append(i * i)
    return result

def squares_comp(n):
    return [i * i for i in range(n)]

print("\n=== Loop version ===")
dis.dis(squares_loop)

print("\n=== List comprehension (more efficient bytecode) ===")
dis.dis(squares_comp)

N = 10_000
t_loop = timeit.timeit(lambda: squares_loop(N), number=500)
t_comp = timeit.timeit(lambda: squares_comp(N), number=500)
print(f"\nLoop:        {t_loop:.3f}s")
print(f"Comprehension: {t_comp:.3f}s  ({t_loop/t_comp:.2f}x faster)")


In [ ]:
# ── Global vs local variable lookup ──────────────────────────────────────────
import timeit

GLOBAL_RANGE = range   # global lookup is a dict lookup each time

def sum_global(n):
    total = 0
    for i in range(n):     # 'range' looked up in globals dict every iteration
        total += i
    return total

def sum_local(n):
    total = 0
    _range = range         # hoist global to local slot (array index — O(1))
    for i in _range(n):
        total += i
    return total

N = 1_000_000
t_g = timeit.timeit(lambda: sum_global(N), number=5)
t_l = timeit.timeit(lambda: sum_local(N),  number=5)
print(f"Global lookup: {t_g:.3f}s")
print(f"Local  lookup: {t_l:.3f}s  ({t_g/t_l:.2f}x faster)")

# ── Inspect .pyc cache ────────────────────────────────────────────────────────
import py_compile, marshal, importlib.util, tempfile, os

with tempfile.NamedTemporaryFile(suffix='.py', delete=False, mode='w') as f:
    f.write("def hello(): return 'world'\n")
    py_path = f.name

pyc_path = py_compile.compile(py_path, cfile=py_path + 'c')
size_py  = os.path.getsize(py_path)
size_pyc = os.path.getsize(pyc_path)
print(f"\n.py  size: {size_py} bytes")
print(f".pyc size: {size_pyc} bytes  (includes magic number, timestamp, size, marshalled code)")

os.unlink(py_path)
os.unlink(pyc_path)


---
## 📊 Topic 21: Profiling & Performance Optimization
`────────────────────────────────────────────────────────────`


### What is it?
**Optimization without profiling is guesswork.**
Python provides multiple profiling tools:

| Tool | What it measures | Overhead |
|------|-----------------|----------|
| `cProfile` | Function call counts & time | Low |
| `tracemalloc` | Memory allocations by line | Medium |
| `timeit` | Micro-benchmark a snippet | Minimal |
| `line_profiler` | Line-by-line timing | High (dev only) |
| `py-spy` | Sampling profiler (no code change) | Very low |

### The workflow
```
1. Measure (cProfile)  →  2. Find hotspot  →  3. Benchmark change (timeit)
→  4. Apply fix  →  5. Verify speedup  →  repeat
```


In [ ]:
import cProfile, pstats, io, tracemalloc, functools, timeit

# ── 1. cProfile ────────────────────────────────────────────────────────────────

def slow_process(n):
    """Deliberately inefficient — uses list for membership test."""
    data = list(range(n))
    return sum(x for x in data if x in data[: n // 2])   # O(n²)

def profile_func(func, *args, top_n=8):
    pr = cProfile.Profile()
    pr.enable()
    result = func(*args)
    pr.disable()
    buf = io.StringIO()
    pstats.Stats(pr, stream=buf).sort_stats('cumulative').print_stats(top_n)
    print(buf.getvalue())
    return result

print("=== cProfile output ===")
profile_func(slow_process, 5_000)


# ── 2. tracemalloc — memory by line ────────────────────────────────────────────
tracemalloc.start()

def allocate_lots():
    return {i: [j**2 for j in range(100)] for i in range(500)}

data = allocate_lots()
snap = tracemalloc.take_snapshot()
print("\n=== Top memory lines (tracemalloc) ===")
for stat in snap.statistics('lineno')[:3]:
    print(f"  {stat}")
tracemalloc.stop()
del data


In [ ]:
import timeit, functools

# ── 3. Key optimization patterns ──────────────────────────────────────────────

print("=== Pattern A: set vs list for membership ===")
N    = 100_000
lst  = list(range(N))
st   = set(lst)

t_list = timeit.timeit(lambda: N-1 in lst, number=10_000)
t_set  = timeit.timeit(lambda: N-1 in st,  number=10_000)
print(f"  list O(n): {t_list*1000:.2f}ms")
print(f"  set  O(1): {t_set*1000:.4f}ms  ({t_list/t_set:.0f}x faster)")


print("\n=== Pattern B: lru_cache for pure functions ===")
def fib_slow(n):
    if n < 2: return n
    return fib_slow(n-1) + fib_slow(n-2)

@functools.lru_cache(maxsize=None)
def fib_fast(n):
    if n < 2: return n
    return fib_fast(n-1) + fib_fast(n-2)

t_slow = timeit.timeit(lambda: fib_slow(25), number=10)
t_fast = timeit.timeit(lambda: fib_fast(25), number=10)
print(f"  Without cache: {t_slow:.4f}s")
print(f"  With cache:    {t_fast:.6f}s  ({t_slow/t_fast:.0f}x faster)")


print("\n=== Pattern C: str.join vs concatenation ===")
words = ['hello'] * 1000
t_concat = timeit.timeit(lambda: ''.join(['x' + w for w in words]), number=1000)
t_join   = timeit.timeit(lambda: ' '.join(words), number=1000)
print(f"  concat loop: {t_concat*1000:.2f}ms")
print(f"  str.join:    {t_join*1000:.3f}ms  ({t_concat/t_join:.1f}x faster)")


print("\n=== Pattern D: list comprehension vs map+lambda ===")
data = list(range(10_000))
t_map  = timeit.timeit(lambda: list(map(lambda x: x*2, data)), number=1000)
t_comp = timeit.timeit(lambda: [x*2 for x in data],            number=1000)
print(f"  map+lambda:  {t_map*1000:.2f}ms")
print(f"  list comp:   {t_comp*1000:.2f}ms  ({t_map/t_comp:.2f}x faster)")

print("\n✅ Golden Rule: Measure → Find → Fix → Verify → Repeat")


---

## 🎓 Summary & Quick Reference

| Topic | Key Takeaway | Best Use Case |
|-------|-------------|---------------|
| **Metaclasses** | Class of a class; intercepts class creation | Auto-registration, ORM field discovery |
| **Descriptors** | Reusable attribute logic via `__get__`/`__set__` | Validated fields, computed properties |
| **Context Managers** | Guaranteed setup/teardown via `with` | DB transactions, file locking |
| **Decorators** | Wrap functions without modifying them | Retry, caching, auth, logging |
| **Generators** | Lazy evaluation, O(1) memory pipelines | Streaming ETL, large datasets |
| **Asyncio** | Cooperative concurrency for I/O | High-concurrency servers, scrapers |
| **Multithreading** | I/O overlap in same process | Parallel downloads, background tasks |
| **Multiprocessing** | True CPU parallelism, no GIL | Batch computation, ML preprocessing |
| **GIL** | Only one thread runs Python at a time | Explains why threads ≠ speedup for CPU |
| **Memory / GC** | Refcounting + cyclic GC | Long-running services, leak detection |
| **Weak References** | Hold ref without preventing GC | Caches, observer patterns |
| **`__slots__`** | Replace `__dict__` with fixed array | High-volume objects, tight memory |
| **ABC** | Enforce interface contracts | Plugin systems, adapters |
| **Monkey Patching** | Replace attrs at runtime | Testing mocks, instrumentation |
| **Closures** | Capture enclosing scope | Factory functions, stateful callbacks |
| **Introspection** | Examine types/attrs/signatures at runtime | Frameworks, CLI generators |
| **Dynamic Attrs** | `__getattr__` for missing attrs | ORM lazy-loading, proxies |
| **Dunder Methods** | Participate in Python syntax | Domain types, DSLs |
| **C Extensions** | Compiled code, bypass GIL | Hotspot optimisation, wrappers |
| **Bytecode** | What CPython actually executes | Performance analysis, tooling |
| **Profiling** | Measure before optimising | Every performance task |

---
*Generated for Google Colab · All examples use Python stdlib only · No pip installs required*
